<a href="https://colab.research.google.com/github/khusheedhanuka/machine_learning/blob/main/VanillaRNN_and_BiRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
from keras.layers import Activation,Embedding,Flatten
from keras.layers import Dropout
from keras.optimizers import Adam
import pandas as pd


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d yasserh/imdb-movie-ratings-sentiment-analysis

imdb-movie-ratings-sentiment-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-movie-ratings-sentiment-analysis.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
data=pd.read_csv('/content/movie.csv')



In [ ]:
data


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [ ]:
data.columns

Index(['text', 'label'], dtype='object')

In [ ]:
X=data['text']
y=data['label']


In [ ]:
X

0        I grew up (b. 1965) watching and loving the Th...
1        When I put this movie in my DVD player, and sa...
2        Why do people who do not know what a particula...
3        Even though I have great interest in Biblical ...
4        Im a die hard Dads Army fan and nothing will e...
                               ...                        
39995    "Western Union" is something of a forgotten cl...
39996    This movie is an incredible piece of work. It ...
39997    My wife and I watched this movie because we pl...
39998    When I first watched Flatliners, I was amazed....
39999    Why would this film be so good, but only gross...
Name: text, Length: 40000, dtype: object

In [ ]:
X.shape

(40000,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)

X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)


vocab=len(tokenizer.word_index)+1






In [ ]:
print(X_train[6])


[15, 143, 12, 233, 69, 910, 9, 46, 6, 3, 8, 373, 117, 143, 12, 98, 23, 5, 5, 26, 27, 4, 143, 98, 7, 7, 250, 321, 4, 3, 246, 83, 17, 47, 650, 514, 1, 352, 18, 12, 149, 1, 241, 4, 12, 3, 19, 37, 11, 45, 74, 1, 64, 6, 405, 214, 96, 2, 5, 99, 144, 83, 25, 11, 19, 5, 1, 18, 143, 539, 4, 83, 23, 1, 12, 355, 5, 439, 104, 23, 808, 5, 1, 167, 581, 2, 696, 12, 1, 104, 18, 83, 459, 5, 25, 12, 46, 23, 228, 50, 76, 104, 72, 49, 653, 213, 18, 92, 213, 82, 7, 7, 87, 122, 1, 19, 6, 96, 8, 3, 246, 704, 4, 80, 24, 114, 100, 725, 151, 83, 5, 519, 37, 28, 111, 312, 1, 4, 263, 1, 868, 6, 208, 31, 3, 129, 61, 10, 78, 131, 13, 246, 1, 19, 310, 5, 3, 169, 4, 896, 749, 175, 1, 320, 12, 1, 78, 221, 53, 306, 8, 3, 50, 2, 50, 70, 189, 42, 95, 191, 9, 5, 20, 143, 175, 134, 1, 5, 47, 4, 24, 2, 392, 68, 945, 37, 858, 9, 289, 337, 18, 272, 73, 142, 538, 1, 19, 14, 137, 9, 213]


In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen=200

X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)







In [ ]:
from keras.utils import to_categorical

num_classes=2
y_train=to_categorical(y_train,num_classes)
y_test=to_categorical(y_test,num_classes)

y_train[0]

array([1., 0.], dtype=float32)

In [ ]:
model=Sequential()
model.add(SimpleRNN(200,input_shape=(maxlen,1),return_sequences=False))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer= 'adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 200)               40400     
                                                                 
 dense_1 (Dense)             (None, 2)                 402       
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
Total params: 40802 (159.38 KB)
Trainable params: 40802 (159.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Epoch 1/30
1000/1000 [==============================] - 49s 48ms/step - loss: 0.7119 - accuracy: 0.5031 - val_loss: 0.6943 - val_accuracy: 0.5015
Epoch 2/30
1000/1000 [==============================] - 47s 47ms/step - loss: 0.7033 - accuracy: 0.5042 - val_loss: 0.7012 - val_accuracy: 0.5090
Epoch 3/30
1000/1000 [==============================] - 45s 45ms/step - loss: 0.7032 - accuracy: 0.5036 - val_loss: 0.6966 - val_accuracy: 0.5100
Epoch 4/30
1000/1000 [==============================] - 46s 46ms/step - loss: 0.7021 - accuracy: 0.5036 - val_loss: 0.6922 - val_accuracy: 0.5206
Epoch 5/30
1000/1000 [==============================] - 48s 48ms/step - loss: 0.6990 - accuracy: 0.5070 - val_loss: 0.7014 - val_accuracy: 0.5117
Epoch 6/30
1000/1000 [==============================] - 47s 47ms/step - loss: 0.7023 - accuracy: 0.5017 - val_loss: 0.6962 - val_accuracy: 0.4882
Epoch 7/30
1000/1000 [==============================] - 46s 46ms/step - loss: 0.7023 - accuracy: 0.5004 - val_loss: 0.6940 -

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

y_pred=model.predict(X_test)

y_test_=np.argmax(y_test,axis=1)
y_pred_=np.argmax(y_pred,axis=1)
# print(y_test_.shape)
# print(y_pred.shape)
# # y_pred.reshape(8000,)
# print(y_test_)
print(accuracy_score(y_test_,y_pred_))





250/250 [==============================] - 4s 15ms/step
0.487375


In [ ]:
# model2 = Sequential()
# model2.add(Embedding(input_dim=200, output_dim=32, input_length=X_train.shape[1]))

# model2.add(SimpleRNN(64,input_shape=(maxlen,1) ,return_sequences=True))

# backward_rnn = SimpleRNN(64, return_sequences=True, go_backwards=True)
# model2.add(backward_rnn)

# model2.add(Flatten())


# model2.add(Dense(2, activation='sigmoid'))

# model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model2.summary()

In [ ]:
# y_train_binary = np.argmax(y_train, axis=1)

In [ ]:
# model2.fit(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

In [ ]:
X_train.shape


(32000, 200)

In [ ]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [ ]:
from tensorflow.keras.models import load_model, save_model


# model2.save('/content/drive/MyDrive/my_model.h5')


model2=load_model('/content/drive/MyDrive/my_model.h5')

In [ ]:
y_pred=model2.predict(X_test)

y_test_=np.argmax(y_test,axis=1)
y_pred_=np.argmax(y_pred,axis=1)

print(accuracy_score(y_test_,y_pred_))

250/250 [==============================] - 10s 40ms/step
0.71225
